In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("hokarami/TEDAM3")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

In [3]:
runs_df

,summary,config,name
0,{'Train-pred_label/f1-binary': 0.8061698990586...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[mc2]TEDA__shpmarklabel-1144040
1,"{'Test-pred_label/AUPRC': 0.4128057281534922, ...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[mc2]DA__label-1144040
2,{'Test-pred_label/f1-binary': 0.42560175054704...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[mc2]DA__label-1143514
3,"{'Valid-NextTime/msae': 1.3548336330689672, 'T...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[mc2]TEDA__shpmarklabel-1143514
4,"{'Valid-pred_label/ACC': 0.6551810865191147, '...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[mc2]DA__label-1143514
...,...,...,...
376,"{'Test-pred_label/ACC': 0.7491666666666666, 'T...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[sseft1]DA__label-1085186-none-state1-per100
377,"{'_step': 60, 'time-Time/train_epoch': 0.08596...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[wd4]DA__label-1084470-none-state1-per20
378,"{'_runtime': 176.44182109832764, 'Obj': 0, 'Va...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[wd2]DA__label-1084470-none-state1-per20
379,{'Best-Test-pred_label/AUPRC': 0.2760899315999...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[wd3]DA__label-1084470-none-state1-per20


In [4]:

runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

dict_keys(['Train-pred_label/f1-binary', 'Valid-NextType(ML)/auc-ovo-weighted', '_step', 'Test-NextTime/msae', 'Train-NextType(ML)/precision-weighted', 'Train-NextTime/msae', 'Valid-CIF/NLL-#events', 'time-Time/train_epoch', 'Valid-pred_label/AUROC', 'Best-Test-pred_label/AUROC', 'Best-Valid-pred_label/f1-binary', 'Test-pred_label/precision-binary', 'Train-NextType(ML)/recall-weighted', 'Valid-pred_label/AUPRC', 'Train-NextType(ML)/auc-ovo-weighted', 'Valid-CIF/NLL', 'Valid-NextTime/rmse_norm', 'Best-Test-pred_label/AUPRC', 'Valid-NextType(ML)/f1-weighted', 'Test-NextType(ML)/recall-weighted', 'time-Time/valid_epoch', 'Test-NextTime/rmse_norm', 'Valid-pred_label/f1-binary', 'Test-NextType(ML)/f1-weighted', 'Best-Test-pred_label/f1-binary', 'Test-NextType(ML)/auc-ovo-weighted', 'Train-NextType(ML)/auc-PR-weighted', '_runtime', 'Test-pred_label/AUROC', 'Train-NextTime/RMSE', 'Valid-NextTime/msae', 'Test-pred_label/loss', 'Train-NextTime/rmse_norm', 'Test-pred_label/recall-binary', 'Test-

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

In [5]:
runs_df.iloc[0].config.keys()

d_name = [x for x in runs_df.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in runs_df.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in runs_df.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = runs_df.name
df = df[['dataset','setting','event_enc','test_center','split']+['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']]
df


i_by =['dataset','setting','event_enc']

a0 = df.groupby(i_by)['Best-Test-pred_label/f1-binary'].size().rename('runs')

a1 = df.groupby(i_by)['Best-Test-pred_label/f1-binary'].mean().round(3).rename('mean')
a2 = df.groupby(i_by)['Best-Test-pred_label/f1-binary'].std().round(3).rename('std')
a4 = df.groupby(i_by)['Best-Test-pred_label/f1-binary'].apply(lambda x:x.round(3).tolist()).rename('cat')   # x.round(3).tolist()

df2 = pd.concat([a0,a1,a2],axis=1)
df2

df2.to_csv("aaa.csv")

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

,dataset,setting,event_enc,test_center,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC
0,/scratch/hokarami/data_tedam/p12/,mc2,1,2,0,0.452055,0.432469
1,/scratch/hokarami/data_tedam/p12/,mc2,0,2,0,0.433526,0.417564
2,/scratch/hokarami/data_tedam/p12/,mc2,0,0,0,0.425602,0.410832
3,/scratch/hokarami/data_tedam/p12/,mc2,1,0,0,0.439211,0.439053
4,/scratch/hokarami/data_tedam/p12/,mc2,0,1,0,0.425156,0.423649
...,...,...,...,...,...,...,...
376,NaN,NaN,0,NaN,NaN,0.497482,0.535767
377,NaN,NaN,0,NaN,NaN,0.400000,0.327426
378,NaN,NaN,0,NaN,NaN,0.391650,0.383884
379,NaN,NaN,0,NaN,NaN,0.344828,0.276090


runs   mean    std
dataset                           setting event_enc                    
/scratch/hokarami/data_tedam/p12/ mc1     0            15  0.492  0.028
                                          1            15  0.492  0.026
                                  mc2     0             3  0.428  0.005
                                          1             3  0.445  0.007
                                  sc      0            15  0.405  0.035
                                          1            15  0.425  0.026
/scratch/hokarami/data_tedam/p19/ mc1     0            10  0.728  0.061
                                          1            10  0.711  0.038
                                  mc2     0            10  0.533  0.095
                                          1             5  0.469  0.050
                                  sc      0            10  0.619  0.039
                                          1            10  0.632  0.030

In [18]:
temp = df2.reset_index()[df2.reset_index().dataset=='/scratch/hokarami/data_tedam/p12/']
fig = px.histogram(temp, x='setting' ,y='mean',color='event_enc', barmode='group')
fig.show()